In [1]:
#version de python
import sys
sys.version

'3.11.11 | packaged by conda-forge | (main, Dec  5 2024, 14:06:23) [MSC v.1942 64 bit (AMD64)]'

In [2]:
#version de pycaret
import pycaret
print(pycaret.__version__)

3.2.0


In [3]:
#version de mlflow
import mlflow
print(mlflow.__version__)

1.30.1


In [4]:
#changer de dossier
import os
os.chdir("C:/Users/lucie/venv")

In [5]:
import mlflow
import time
#importation de l'outil d'expérimentation de pycaret
from pycaret.classification import ClassificationExperiment
#mlflow.set_tracking_uri("file:///C:/Users/lucie/venv/mlruns")
mlflow.set_tracking_uri("http://127.0.0.1:5000")


In [6]:
#chargement des données 
import pandas as pd
data = pd.read_csv('Loan_Data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               10000 non-null  int64  
 1   credit_lines_outstanding  10000 non-null  int64  
 2   loan_amt_outstanding      10000 non-null  float64
 3   total_debt_outstanding    10000 non-null  float64
 4   income                    10000 non-null  float64
 5   years_employed            10000 non-null  int64  
 6   fico_score                10000 non-null  int64  
 7   default                   10000 non-null  int64  
dtypes: float64(3), int64(5)
memory usage: 625.1 KB


In [7]:
import time
import mlflow
from pycaret.classification import ClassificationExperiment

# S'assurer que toute session MLflow précédente est fermée
if mlflow.active_run():
    mlflow.end_run()

# Démarrer la session primaire avec suivi MLflow
with mlflow.start_run(run_name="recherche_modele"):
    start_time = time.time()

    # Initialiser et configurer l'expérience
    session_prim = ClassificationExperiment()
    session_prim.setup(data, normalize=True, target='default', train_size=0.7,
                       data_split_stratify=True, fold=5, session_id=0,
                       log_experiment=True, experiment_name="recherche_modele")
    
    duration = time.time() - start_time
    mlflow.log_param("duration", duration)
    
    # Afficher les algorithmes disponibles
    algos = session_prim.models()
    print(algos)

    # Initialisation d'une liste vide avant la boucle
    top_models = []

    # Comparer et enregistrer la durée pour chaque modèle
    model_list = ['lr', 'nb', 'dt', 'rf', 'svm', 'lda']
    model_durations = {}

    for model in model_list:
        start_time = time.time()
        mdl = session_prim.create_model(model)
        duration = time.time() - start_time
        model_durations[model] = duration
        mlflow.log_param(f"duration_{model}", duration)
        top_models.append(mdl)  # Correction : on passe 'mdl' comme argument

        # Comparer et sélectionner les meilleurs modèles
        best_model = session_prim.compare_models(sort='Accuracy', include=model_list, verbose=True)
        print("Durées d'entraînement par modèle :", model_durations)
        # Extraire les résultats
        results = session_prim.pull()
        print(results)
        
        import os
        # Passer à MLflow 2.20.2
        os.system("conda activate theo1_env")
        #  Enregistrement du modèle "recherche-model" dans MLflow
        import mlflow.sklearn
        if hasattr(best_model, "estimators_"):
           best_model = best_model.estimators_[0]  # Prendre le premier modèle du pipeline
           mlflow.sklearn.log_model(best_model, "recherche-model")
           # Ajouter au registre MLflow
           mlflow.register_model(f"runs:/{mlflow.active_run().info.run_id}/recherche-model", "recherche-model")
        
           # Revenir à MLflow 1.30.1
           os.system("conda activate theo_env")
# Fin explicite de la session (facultatif ici car le 'with' gère la fermeture)
mlflow.end_run()


,Description,Value
0,Session id,0
1,Target,default
2,Target type,Binary
3,Original data shape,"(10000, 8)"
4,Transformed data shape,"(10000, 8)"
5,Transformed train set shape,"(7000, 8)"
6,Transformed test set shape,"(3000, 8)"
7,Numeric features,7
8,Preprocess,True
9,Imputation type,simple


2025/02/25 08:49:26 INFO mlflow.tracking.fluent: Experiment with name 'recherche_modele' does not exist. Creating a new experiment.


                                     Name  \
ID                                          
lr                    Logistic Regression   
knn                K Neighbors Classifier   
nb                            Naive Bayes   
dt               Decision Tree Classifier   
svm                   SVM - Linear Kernel   
rbfsvm                SVM - Radial Kernel   
gpc           Gaussian Process Classifier   
mlp                        MLP Classifier   
ridge                    Ridge Classifier   
rf               Random Forest Classifier   
qda       Quadratic Discriminant Analysis   
ada                  Ada Boost Classifier   
gbc          Gradient Boosting Classifier   
lda          Linear Discriminant Analysis   
et                 Extra Trees Classifier   
lightgbm  Light Gradient Boosting Machine   
dummy                    Dummy Classifier   

                                                  Reference  Turbo  
ID                                                                  
lr    

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9979,1.0000,0.9923,0.9961,0.9942,0.9929,0.9929
1,0.9936,1.0000,0.9691,0.9960,0.9824,0.9785,0.9786
2,0.9993,1.0000,0.9961,1.0000,0.9981,0.9976,0.9976
3,0.9979,1.0000,0.9884,1.0000,0.9942,0.9929,0.9929
4,0.9993,1.0000,0.9962,1.0000,0.9981,0.9976,0.9976
Mean,0.9976,1.0000,0.9884,0.9984,0.9934,0.9919,0.9919
Std,0.0021,0.0000,0.0101,0.0019,0.0058,0.0070,0.0070


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9976,1.0000,0.9884,0.9984,0.9934,0.9919,0.9919,1.1420
svm,SVM - Linear Kernel,0.9969,0.0000,0.9892,0.9939,0.9914,0.9895,0.9896,0.0360
dt,Decision Tree Classifier,0.9943,0.9914,0.9869,0.9824,0.9846,0.9811,0.9811,0.0260
rf,Random Forest Classifier,0.9939,0.9998,0.9807,0.9861,0.9834,0.9796,0.9796,0.2700
lda,Linear Discriminant Analysis,0.9787,0.9997,0.9992,0.8976,0.9457,0.9325,0.9346,0.0320
nb,Naive Bayes,0.9721,0.9973,0.9722,0.8880,0.9282,0.9109,0.9124,0.0260


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9976,1.0000,0.9884,0.9984,0.9934,0.9919,0.9919,0.0260
svm,SVM - Linear Kernel,0.9969,0.0000,0.9892,0.9939,0.9914,0.9895,0.9896,0.0240
dt,Decision Tree Classifier,0.9943,0.9914,0.9869,0.9824,0.9846,0.9811,0.9811,0.0240
rf,Random Forest Classifier,0.9939,0.9998,0.9807,0.9861,0.9834,0.9796,0.9796,0.2740
lda,Linear Discriminant Analysis,0.9787,0.9997,0.9992,0.8976,0.9457,0.9325,0.9346,0.0260
nb,Naive Bayes,0.9721,0.9973,0.9722,0.8880,0.9282,0.9109,0.9124,0.0220


Durées d'entraînement par modèle : {'lr': 10.157962560653687, 'nb': 1.637826919555664, 'dt': 2.0386698246002197, 'rf': 4.752106666564941, 'svm': 2.6197047233581543}
                            Model  Accuracy     AUC  Recall   Prec.      F1  \
lr            Logistic Regression    0.9976  1.0000  0.9884  0.9984  0.9934   
svm           SVM - Linear Kernel    0.9969  0.0000  0.9892  0.9939  0.9914   
dt       Decision Tree Classifier    0.9943  0.9914  0.9869  0.9824  0.9846   
rf       Random Forest Classifier    0.9939  0.9998  0.9807  0.9861  0.9834   
lda  Linear Discriminant Analysis    0.9787  0.9997  0.9992  0.8976  0.9457   
nb                    Naive Bayes    0.9721  0.9973  0.9722  0.8880  0.9282   

      Kappa     MCC  TT (Sec)  
lr   0.9919  0.9919     0.026  
svm  0.9895  0.9896     0.024  
dt   0.9811  0.9811     0.024  
rf   0.9796  0.9796     0.274  
lda  0.9325  0.9346     0.026  
nb   0.9109  0.9124     0.022  


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9843,0.9998,1.0000,0.9217,0.9593,0.9495,0.9508
1,0.9764,0.9996,0.9961,0.8897,0.9399,0.9253,0.9276
2,0.9750,0.9996,1.0000,0.8810,0.9367,0.9212,0.9241
3,0.9786,0.9998,1.0000,0.8962,0.9453,0.9320,0.9341
4,0.9793,0.9998,1.0000,0.8997,0.9472,0.9343,0.9364
Mean,0.9787,0.9997,0.9992,0.8976,0.9457,0.9325,0.9346
Std,0.0032,0.0001,0.0015,0.0136,0.0078,0.0097,0.0092


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9976,1.0000,0.9884,0.9984,0.9934,0.9919,0.9919,0.0260
svm,SVM - Linear Kernel,0.9969,0.0000,0.9892,0.9939,0.9914,0.9895,0.9896,0.0220
dt,Decision Tree Classifier,0.9943,0.9914,0.9869,0.9824,0.9846,0.9811,0.9811,0.0280
rf,Random Forest Classifier,0.9939,0.9998,0.9807,0.9861,0.9834,0.9796,0.9796,0.2400
lda,Linear Discriminant Analysis,0.9787,0.9997,0.9992,0.8976,0.9457,0.9325,0.9346,0.0240
nb,Naive Bayes,0.9721,0.9973,0.9722,0.8880,0.9282,0.9109,0.9124,0.0220


Durées d'entraînement par modèle : {'lr': 10.157962560653687, 'nb': 1.637826919555664, 'dt': 2.0386698246002197, 'rf': 4.752106666564941, 'svm': 2.6197047233581543, 'lda': 3.470503807067871}
                            Model  Accuracy     AUC  Recall   Prec.      F1  \
lr            Logistic Regression    0.9976  1.0000  0.9884  0.9984  0.9934   
svm           SVM - Linear Kernel    0.9969  0.0000  0.9892  0.9939  0.9914   
dt       Decision Tree Classifier    0.9943  0.9914  0.9869  0.9824  0.9846   
rf       Random Forest Classifier    0.9939  0.9998  0.9807  0.9861  0.9834   
lda  Linear Discriminant Analysis    0.9787  0.9997  0.9992  0.8976  0.9457   
nb                    Naive Bayes    0.9721  0.9973  0.9722  0.8880  0.9282   

      Kappa     MCC  TT (Sec)  
lr   0.9919  0.9919     0.026  
svm  0.9895  0.9896     0.022  
dt   0.9811  0.9811     0.028  
rf   0.9796  0.9796     0.240  
lda  0.9325  0.9346     0.024  
nb   0.9109  0.9124     0.022  
